In [0]:
import pandas as pd

In [2]:
data=pd.read_csv("zoo.data")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
5,buffalo,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
6,calf,1,0,0,1,0,0,0,1,1,1,0,0,4,1,1,1,1
7,carp,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,4
8,catfish,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
9,cavy,1,0,0,1,0,0,0,1,1,1,0,0,4,0,1,0,1


In [3]:
data.shape

(101, 18)

In [4]:
x_data=data.iloc[:,:-1]
x_data.shape

(101, 17)

In [5]:
y_data=data.iloc[:,-1:]
y_data.shape

(101, 1)

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=21)


In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(80, 17)
(21, 17)
(80, 1)
(21, 1)


In [8]:
test_animal_names=x_test['animal_name']
print(test_animal_names.shape)
x_train=x_train.iloc[:,1:]
print(x_train.shape)
x_test=x_test.iloc[:,1:]
print(x_test.shape)

(21,)
(80, 16)
(21, 16)


In [0]:
import tensorflow as tf
X=tf.placeholder(tf.float32,[None,16])
y_true=tf.placeholder(tf.int32,[None,1])

In [0]:
y_one_hot_encoded=tf.one_hot(y_true,7)
y_one_hot_encoded=tf.reshape(y_one_hot_encoded,[-1,7])

In [11]:
w1=tf.Variable(tf.random_normal([16,20],seed=3),name="weight1")
b1=tf.Variable(tf.random_normal([20],seed=3),name='bias1')

w2=tf.Variable(tf.random_normal([20,7],seed=3),name="weight2")
b2=tf.Variable(tf.random_normal([7],seed=3),name='bias2')

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
input_layer=tf.add(tf.matmul(X,w1),b1)
input_layer=tf.nn.relu(input_layer)
input_layer=tf.nn.dropout(input_layer,keep_prob=0.8)

out_layer=tf.add(tf.matmul(input_layer,w2),b2)
out_layer=tf.nn.relu(out_layer)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_one_hot_encoded,logits=out_layer))

In [0]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [0]:
prediction=tf.argmax(tf.nn.softmax(out_layer),1)
correct_prediction=tf.equal(prediction,tf.argmax(y_one_hot_encoded,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [16]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  feed_dict1={X:x_train,y_true:y_train}
  for step in range(10000):
    sess.run(optimizer,feed_dict=feed_dict1)
    if(step%100==0):
      loss,acc=sess.run([cost,accuracy],feed_dict=feed_dict1)
      print("step{:5}  loss:{:.3f} acc:{:.2%}".format(step,loss,acc))
      
  train_acc=sess.run(accuracy,feed_dict=feed_dict1)
  test_acc,y_pred,correct_results=sess.run([accuracy,prediction,correct_prediction],feed_dict={X:x_test,y_true:y_test})
  print("train_accuracy:",train_acc*100)
  print("test accuracy:",test_acc*100)

step    0  loss:19.727 acc:6.25%
step  100  loss:0.495 acc:81.25%
step  200  loss:0.299 acc:85.00%
step  300  loss:0.209 acc:90.00%
step  400  loss:0.227 acc:92.50%
step  500  loss:0.085 acc:97.50%
step  600  loss:0.038 acc:96.25%
step  700  loss:0.009 acc:97.50%
step  800  loss:0.067 acc:96.25%
step  900  loss:0.127 acc:93.75%
step 1000  loss:0.103 acc:96.25%
step 1100  loss:0.057 acc:97.50%
step 1200  loss:0.067 acc:95.00%
step 1300  loss:0.081 acc:96.25%
step 1400  loss:0.109 acc:92.50%
step 1500  loss:0.051 acc:98.75%
step 1600  loss:0.069 acc:97.50%
step 1700  loss:0.059 acc:97.50%
step 1800  loss:0.011 acc:98.75%
step 1900  loss:0.033 acc:97.50%
step 2000  loss:0.054 acc:98.75%
step 2100  loss:0.052 acc:96.25%
step 2200  loss:0.004 acc:98.75%
step 2300  loss:0.043 acc:93.75%
step 2400  loss:0.028 acc:97.50%
step 2500  loss:0.058 acc:97.50%
step 2600  loss:0.116 acc:95.00%
step 2700  loss:0.026 acc:98.75%
step 2800  loss:0.116 acc:93.75%
step 2900  loss:0.083 acc:95.00%
step 3000 

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
print("confuson_matrix:\n",confusion_matrix(y_test,y_pred))
print("classification_report\n\n",classification_report(y_test,y_pred))

confuson_matrix:
 [[ 0  0  0  0  0  0]
 [ 0 10  0  0  0  0]
 [ 0  0  5  0  0  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  1  0]
 [ 3  0  0  1  0  0]]
classification_report

               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       0.50      1.00      0.67         1
           6       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         4

   micro avg       0.81      0.81      0.81        21
   macro avg       0.58      0.67      0.61        21
weighted avg       0.79      0.81      0.79        21



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
checking_model_prediction_on_test_data=pd.DataFrame()
checking_model_prediction_on_test_data['test_animal_names']=test_animal_names
checking_model_prediction_on_test_data['prediction']=y_predict
checking_model_prediction_on_test_data['actual']=y_test
checking_model_prediction_on_test_data['corrrectness']=correct_results

NameError: ignored

In [0]:
checking_model_prediction_on_test_data